In [ ]:
import functools
import time
import math
import tensorflow as tf
from tensorflow import keras
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

Defining Constants

In [ ]:
DEVICE = 'GPU/:0'

DATASET_DIR = '../../../datasets/Dataset-IoT/'
NETFLOW_DIR = DATASET_DIR + 'MC/NetFlow/'

# MC_I_FIRST: Has infected data by Hajime, Aidra and BashLite botnets'
# MC_I_SECOND: Has infected data from Mirai botnets
# MC_I_THIR: Has infected data from Mirai, Doflo, Tsunami and Wroba botnets
# MC_L: Has legitimate data, no infection
MC_L = r'MC_L.csv'

data_set_files = [r'MC_I{}.csv'.format(index) for index in range(1, 4)]
data_set_files.insert(0, r'MC_L.csv')
print (data_set_files)

Loading the Dataset

In [ ]:
legitimate_file_path = NETFLOW_DIR + MC_L

LABEL_COLUMN = 'Label'

#reading data
df = pd.read_csv (legitimate_file_path)

# for file in data_set_files:
#     aux_df = pd.read_csv(NETFLOW_DIR + file)
#     df = pd.concat([df, aux_df], ignore_index=True)

aux_df = pd.read_csv(NETFLOW_DIR + data_set_files[1])
df = pd.concat([df, aux_df], ignore_index=True)

#making the final DataFrame
df = df.drop(df.columns[0], axis=1)
df.describe()

nUniques = df.nunique()
for column, nUnique in zip (df.columns, nUniques):
    if(nUnique == 1):
        df.drop(axis='columns', columns=column, inplace=True)
df

In [ ]:
df.drop(axis='columns', columns=['ts', 'te', 'sa', 'da'], inplace=True)
df

# Encoding the Categorical Columns

In [ ]:
from sklearn import preprocessing

cat_cols, num_cols = df.columns[df.dtypes == 'O'], df.columns[df.dtypes != 'O']
num_cols = num_cols[1:]

categories = [df[column].unique() for column in df[cat_cols]]

categorical_encoder = preprocessing.OrdinalEncoder(categories=categories)
categorical_encoder.fit(df[cat_cols])
df[cat_cols] = categorical_encoder.transform(df[cat_cols])

df

# Splitting the Data

In [ ]:
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

train, test = train_test_split (df, test_size=0.2)
train, val = train_test_split (train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train_labels = np.array (train.pop('Label'))
bool_train_labels = train_labels != 0
val_labels = np.array(val.pop('Label'))
test_labels = np.array (test.pop('Label'))

train_features = np.array(train)
val_features = np.array(val)
test_features = np.array(test)

How imbalanced is the data?

In [ ]:
neg, pos = np.bincount(df['Label'])
print ("Negative Examples: {}".format(neg))
print ("Positive Examples: {}".format(pos))

#dropping the labels columns
df = df.drop(df.columns[0], axis=1)

# Scaling the Data

In [ ]:
#getting the index of the numerical columns
index = [df.columns.get_loc(c) for c in num_cols]
index = np.array(index)

cat_index = [df.columns.get_loc(c) for c in cat_cols]
cat_index = np.array(index)

scaler = StandardScaler()
train_features[:, index] = scaler.fit_transform(train_features[:, index])

val_features[:, index] = scaler.transform(val_features[:, index])

test_features[:, index] = scaler.transform(test_features[:, index])

train_features[:, index] = np.clip(train_features[:, index], -5, 5)
val_features[:, index] = np.clip(val_features[:, index], -5, 5)
test_features[:, index] = np.clip(test_features[:, index], -5, 5)

# Creating the Model

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


def make_model(metrics = METRICS, output_bias=None, hidden_layer_size=32):
  
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
   
    model = keras.Sequential([
        keras.layers.Dense(hidden_layer_size, activation='relu', input_shape=(train_features.shape[-1],)),#, kernel_initializer=initializer),
        keras.layers.Dense(hidden_layer_size, activation='relu'),#, kernel_initializer=initializer),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(hidden_layer_size, activation='relu'),#, kernel_initializer=initializer),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)#, kernel_initializer=initializer)

    ])

    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
    
    return model

In [ ]:
model = make_model()
BATCH_SIZE = 2048

results = model.evaluate (train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print ("Loss: {:0.4f}".format(results[0]))

# Dealing with the Imbalanced data

In [ ]:
total = pos + neg
weight_for_0 = (1/neg)*(total)/2.0
weight_for_1 = (1/pos)*(total)/2.0

class_weight={0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

# Running the Model

In [ ]:
EPOCHS = 10
#we need a bigger batch_size to reduce the effects of the imbalanced data
BATCH_SIZE = 2048

initial_bias = np.log([neg/pos])

with tf.device (DEVICE):
    weighted_model = make_model(output_bias=initial_bias)
    weighted_history = weighted_model.fit(
        train_features,
        train_labels,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(val_features, val_labels),
        class_weight=class_weight)



Plotting function

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_metrics(weighted_history)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [ ]:
val_predictions_weighted = weighted_model.predict(val_features, batch_size=BATCH_SIZE)

weighted_results = weighted_model.evaluate(val_features, val_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(val_labels, val_predictions_weighted)